# Setup

In [1]:
!pip install -q -U google-genai
!pip install -q -U pydantic
!pip install -q -U langchain langchain-google-genai
!pip install -q -U langgraph



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.2/261.2 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.4/463.4 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 76.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.49.1 requires pydantic<2.12,>=2.0, but you have pydantic 2.12.4 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.7/93.7 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.8/473.8 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

In [4]:
import re
from enum import Enum, auto
from typing import Optional, Dict

import os
from google import genai
from pydantic import BaseModel, Field
from typing import Literal, List
from typing import Union, List, Any
from google.genai import types
import time
import json


# Imports Modernos (LangGraph + LangChain Core)
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage, BaseMessage
from langgraph.prebuilt import create_react_agent


In [5]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEYL')
client = genai.Client(api_key=GOOGLE_API_KEY)


# 🏛️ Fase 1: La Era Determinista (Enfoque Tradicional)

En esta primera etapa, viajamos al pasado para ver cómo se programaban los chatbots antes de la IA Generativa. Este enfoque se basa en la **programación imperativa explícita**.

**Conceptos Clave:**
* **Máquina de Estados (State Machine):** El código controla rígidamente en qué paso de la conversación estamos (Esperando -> Pidiendo Nombre -> Confirmando).
* **Regex (Expresiones Regulares):** Buscamos patrones exactos de texto.
* **La Fragilidad:** Si el usuario dice "apartame un sitio" en lugar de "reserva una mesa", el bot falla porque esa palabra no está en su lista permitida.

**Objetivo:** Demostrar la carga cognitiva del programador, quien debía prever cada posible sinónimo y ruta de conversación.

In [9]:
# 1. Definimos los estados posibles de la conversación
# En el pasado, el programador debía "recordar" manualmente en qué paso estaba el usuario.
class BotState(Enum):
    ESPERANDO = auto()
    PIDIENDO_NOMBRE = auto()
    PIDIENDO_PERSONAS = auto()
    CONFIRMACION = auto()

class RestauranteBotLegacy:
    """
    Bot tradicional basado en reglas y expresiones regulares.
    No tiene 'comprensión', solo busca patrones de texto predefinidos.
    """

    def __init__(self):
        self.state: BotState = BotState.ESPERANDO
        self.reserva_data: Dict[str, str] = {}

        # Patrones "Hardcoded" (La fragilidad del sistema)
        # Si el usuario usa una palabra que no está aquí (ej: "apartar lugar"), el bot falla.
        self.patrones_reserva = r"\b(reservar|mesa|lugar|reserva)\b"
        self.patrones_saludo = r"\b(hola|buenos dias|buenas)\b"

    def procesar_input(self, texto_usuario: str) -> str:
        """Procesa el texto basándose en el estado actual y regex."""
        texto_normalizado = texto_usuario.lower().strip()

        # MAQUINA DE ESTADOS: Lógica de flujo explícita

        # 1. Estado: Esperando intención inicial
        if self.state == BotState.ESPERANDO:
            if re.search(self.patrones_reserva, texto_normalizado):
                self.state = BotState.PIDIENDO_NOMBRE
                return "¡Claro! Para gestionar tu reserva, ¿a nombre de quién la anoto?"

            elif re.search(self.patrones_saludo, texto_normalizado):
                return "¡Hola! Bienvenido a 'Python Bistro'. Escribe 'quiero reservar' para comenzar."

            else:
                return "Disculpa, no entendí. Solo puedo gestionar 'reservas' por ahora."

        # 2. Estado: Recopilando Nombre
        elif self.state == BotState.PIDIENDO_NOMBRE:
            self.reserva_data["nombre"] = texto_usuario.title()
            self.state = BotState.PIDIENDO_PERSONAS
            return f"Gracias {self.reserva_data['nombre']}. ¿Para cuántas personas es la mesa?"

        # 3. Estado: Recopilando Cantidad
        elif self.state == BotState.PIDIENDO_PERSONAS:
            # Intentamos extraer un número del texto
            numeros = re.findall(r"\d+", texto_normalizado)

            if numeros:
                self.reserva_data["personas"] = numeros[0]
                self.state = BotState.CONFIRMACION
                return f"Perfecto. Mesa para {self.reserva_data['personas']}. ¿Confirmas la reserva? (si/no)"
            else:
                # Manejo de error rígido
                return "Por favor, escribe el número de personas (ej: 2, 4)."

        # 4. Estado: Confirmación final
        elif self.state == BotState.CONFIRMACION:
            if "si" in texto_normalizado:
                # Reset del estado para la próxima vez
                resumen = f"¡Listo! Reserva confirmada para {self.reserva_data['nombre']} ({self.reserva_data['personas']} personas)."
                self._reset_bot()
                return resumen
            elif "no" in texto_normalizado:
                self._reset_bot()
                return "Entendido, reserva cancelada. ¿En qué más puedo ayudarte?"
            else:
                return "Por favor responde 'si' o 'no'."

        return "Error de estado."

    def _reset_bot(self):
        """Reinicia la memoria del bot."""
        self.state = BotState.ESPERANDO
        self.reserva_data = {}

# --- Bloque de Prueba (Simulación) ---
if __name__ == "__main__":
    bot = RestauranteBotLegacy()

    print("🤖 Bot Tradicional Iniciado (Escribe 'salir' para terminar)\n")

    # Simulamos una conversación simple
    inputs_prueba = [
        "Hola buenas",
        "Quiero reservar una mesa",
        "Juan Perez",
        "Seremos 4 personas",
        "Si, confirmo"
    ]

    for mensaje in inputs_prueba:
        print(f"Usuario: {mensaje}")
        respuesta = bot.procesar_input(mensaje)
        print(f"Bot:     {respuesta}\n")

🤖 Bot Tradicional Iniciado (Escribe 'salir' para terminar)

Usuario: Hola buenas
Bot:     ¡Hola! Bienvenido a 'Python Bistro'. Escribe 'quiero reservar' para comenzar.

Usuario: Quiero reservar una mesa
Bot:     ¡Claro! Para gestionar tu reserva, ¿a nombre de quién la anoto?

Usuario: Juan Perez
Bot:     Gracias Juan Perez. ¿Para cuántas personas es la mesa?

Usuario: Seremos 4 personas
Bot:     Perfecto. Mesa para 4. ¿Confirmas la reserva? (si/no)

Usuario: Si, confirmo
Bot:     ¡Listo! Reserva confirmada para Juan Perez (4 personas).



# 🧠 Fase 2: La Era Semántica (IA Generativa + Estructura)

Damos el salto a la IA con **Gemini 2.5 Flash**. Aquí abandonamos las reglas rígidas (`if/else`) y confiamos en la capacidad del modelo para "entender" la intención.

**Conceptos Clave:**
* **Comprensión Natural:** El modelo entiende contextos complejos, errores ortográficos y jergas.
* **Salida Estructurada (JSON):** Usamos el SDK moderno `google-genai` y **Pydantic** para obligar al modelo a devolver datos limpios y tipados, no solo texto conversacional.
* **La Limitación:** El "Cerebro en una Caja". El modelo puede *decir* que reservó la mesa, pero es una alucinación. No tiene conexión real con la base de datos ni capacidad de acción.

In [10]:
# 1. Definimos el Esquema (Modelos Pydantic)
class DetallesReserva(BaseModel):
    nombre_cliente: str | None = Field(description="Nombre del cliente")
    cantidad_personas: int | None = Field(description="Número de comensales")
    hora: str | None = Field(description="Hora de la reserva")

class RespuestaBot(BaseModel):
    mensaje_para_usuario: str
    estado_reserva: DetallesReserva
    falta_informacion: bool

# 2. Cliente Moderno
#client = genai.Client() -> ejecutado en el Setup

def procesar_input_moderno(texto_usuario: str):
    try:
        # MODELO: Usamos 'gemini-2.5-flash'
        response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=texto_usuario,
            config=types.GenerateContentConfig(
                response_mime_type="application/json",
                response_schema=RespuestaBot,
                system_instruction="Eres un asistente de restaurante. Extrae datos y responde con formato JSON estricto.",
                temperature=0
            )
        )

        # Parseo seguro
        import json
        json_data = json.loads(response.text)
        return RespuestaBot(**json_data)

    except Exception as e:
        return f"Error del Modelo: {e}"

# Prueba
print("Enviando petición a Gemini 2.5 Flash...")
print(procesar_input_moderno("Mesa para 4 personas a las 8pm, soy Luis."))

Enviando petición a Gemini 2.5 Flash...
mensaje_para_usuario='Perfecto Luis, su mesa para 4 personas a las 8pm está confirmada.' estado_reserva=DetallesReserva(nombre_cliente='Luis', cantidad_personas=4, hora='8pm') falta_informacion=False


# 🤖 Fase 3: La Era Agéntica (LangChain + LangGraph + Gemini 2.5)

Esta es la implementación más avanzada (Estado del Arte 2025). Creamos un **Agente Autónomo** capaz de Razonar y Actuar (Patrón ReAct).

**Mejoras Técnicas Implementadas:**
1.  **LangGraph:** Gestionamos la conversación como un grafo de estados, permitiendo bucles de razonamiento.
2.  **Inyección de Estado:** Para máxima compatibilidad, inyectamos las reglas del sistema (`SystemMessage`) directamente en la memoria del agente, haciéndolo robusto frente a cambios de versiones de librerías.
3.  **Parseo Multimodal Robusto:** Implementamos una función de limpieza que normaliza la respuesta de Gemini 2.5, manejando tanto texto plano como respuestas complejas con metadatos.
4.  **Seguridad:** Gestión segura de API Keys mediante `userdata`.

**Objetivo:** Observa cómo el Agente *verifica* disponibilidad antes de *ejecutar* la reserva en la base de datos real.

In [12]:
# ---------------------------------------------------------
# 1. SEGURIDAD Y CONFIGURACIÓN
# ---------------------------------------------------------
try:
    # Recuperamos la clave de forma segura
    MY_API_KEY = userdata.get('GOOGLE_API_KEYL')
except Exception as e:
    raise ValueError("⚠️ Error: No se pudo acceder a 'GOOGLE_API_KEY' en los secretos de Colab. Asegúrate de haberla añadido en el icono de la llave 🔑 a la izquierda.")

# ---------------------------------------------------------
# 2. BASE DE DATOS Y HERRAMIENTAS
# ---------------------------------------------------------
DB_RESERVAS = {
    "reservas": [],
    "mesas_disponibles": ["19:00", "20:00", "21:00"]
}

MENU_CONTEXTO = """
El menú del restaurante 'Python Bistro':
- Especialidad: Risotto de setas con trufa negra (25€).
- Entrada: Ensalada César (12€), Carpaccio de res (15€).
- Postres: Tiramisú casero (8€), Volcán de chocolate (9€).
- Nota: Tenemos opciones sin gluten y veganas bajo petición.
"""

@tool
def consultar_menu(pregunta: str) -> str:
    """Útil para preguntas sobre el menú, platos y precios."""
    return f"Información del menú: {MENU_CONTEXTO}"

@tool
def verificar_disponibilidad(hora: str) -> str:
    """Verifica mesas disponibles (formato HH:MM)."""
    if hora in DB_RESERVAS["mesas_disponibles"]:
        return "Disponible"
    return "No disponible. Horas libres: " + ", ".join(DB_RESERVAS["mesas_disponibles"])

@tool
def guardar_reserva(nombre: str, hora: str, personas: int) -> str:
    """Guarda una reserva. SOLO usar si 'verificar_disponibilidad' retornó 'Disponible'."""
    if hora in DB_RESERVAS["mesas_disponibles"]:
        DB_RESERVAS["mesas_disponibles"].remove(hora)
        DB_RESERVAS["reservas"].append({"nombre": nombre, "hora": hora, "personas": personas})
        return f"ÉXITO: Reserva confirmada para {nombre} a las {hora}."
    else:
        return "ERROR: La mesa ya no está disponible."

tools = [consultar_menu, verificar_disponibilidad, guardar_reserva]

# ---------------------------------------------------------
# 3. CONSTRUCCIÓN DEL AGENTE
# ---------------------------------------------------------

# Inicializamos el modelo pasando la clave EXPLÍCITAMENTE
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    google_api_key=MY_API_KEY  # <--- AQUÍ ESTÁ LA CLAVE
)

# Definimos el Prompt del Sistema
SYSTEM_INSTRUCTIONS= """
Eres el Agente Inteligente del restaurante 'Python Bistro'.
Tienes acceso a herramientas para consultar menú, ver disponibilidad y reservar.

REGLAS:
- Antes de reservar, SIEMPRE verifica la disponibilidad.
- Si no hay mesa, ofrece otra hora.
- Sé amable pero eficiente.
"""

# Usamos 'prompt' que es el estándar actual en LangGraph prebuilt
# para inyectar el System Prompt.
agent_graph = create_react_agent(
    model=llm,
    tools=tools,
    prompt=SYSTEM_INSTRUCTIONS
)

def obtener_texto_limpio(contenido: Union[str, List[Any], Any]) -> str:
    """
    Normaliza la respuesta de Gemini a un string plano.
    Maneja tipos str, list (multimodal) y objetos desconocidos.
    """

    # 1. Caso Base: Si ya es texto, retornamos directo.
    if isinstance(contenido, str):
        return contenido

    # 2. Caso Multimodal: Si es lista, procesamos eficientemente.
    if isinstance(contenido, list):
        texto_partes = []

        for parte in contenido:
            if isinstance(parte, dict):
                # .get('key', '') es más seguro que ['key'] porque no falla si la clave no existe
                texto_partes.append(parte.get('text', ''))

            elif hasattr(parte, 'text'):
                texto_partes.append(str(parte.text))

            else:
                # Si hay un elemento raro en la lista, lo convertimos a string para no perderlo
                texto_partes.append(str(parte))

        # 'join' es la forma pythonica y optimizada de unir texto
        return "".join(texto_partes)

    # 3. Fallback (Red de seguridad):
    # Si llega un entero, None, o un objeto desconocido, lo convertimos a string.
    return str(contenido)

def imprimir_respuesta(respuesta_graph: dict):
    mensajes = respuesta_graph["messages"]
    ultimo_mensaje = mensajes[-1]

    # Usamos la función de limpieza
    texto_final = obtener_texto_limpio(ultimo_mensaje.content)
    print(f"🤖 Agente: {texto_final}\n")
    #print(f"🤖 Agente: {ultimo_mensaje.content}\n")

# ---------------------------------------------------------
# 4. EJECUCIÓN
# ---------------------------------------------------------
if __name__ == "__main__":
    print("🚀 AGENTE INICIADO (Gemini 2.5 + LangGraph + LangChain)\n")

    # ESTRATEGIA DE INYECCIÓN DE ESTADO:
    # Iniciamos el historial poniendo el SystemMessage primero.
    # El modelo leerá esto y sabrá cómo comportarse, sin importar la versión de la librería.
    chat_history: List[BaseMessage] = []

    # --- CASO 1 ---
    print("--- CASO 1: Consulta de Menú ---")
    input_1 = "Hola, ¿tienen opciones veganas?"
    print(f"👤 Usuario: {input_1}")

    chat_history.append(HumanMessage(content=input_1))

    # Invocamos al agente pasando todo el historial
    response_1 = agent_graph.invoke({"messages": chat_history})

    chat_history = response_1["messages"]
    imprimir_respuesta(response_1)

    # --- CASO 2 ---
    print("--- CASO 2: Intento Reserva (22:00 Lleno) ---")
    input_2 = "Quiero reservar para 4 personas a las 22:00."
    print(f"👤 Usuario: {input_2}")

    chat_history.append(HumanMessage(content=input_2))
    response_2 = agent_graph.invoke({"messages": chat_history})

    chat_history = response_2["messages"]
    imprimir_respuesta(response_2)

    # --- CASO 3 ---
    print("--- CASO 3: Corrección (21:00 Disponible) ---")
    input_3 = "Entendido, entonces a las 21:00 a nombre de Ana."
    print(f"👤 Usuario: {input_3}")

    chat_history.append(HumanMessage(content=input_3))
    response_3 = agent_graph.invoke({"messages": chat_history})

    chat_history = response_3["messages"]
    imprimir_respuesta(response_3)

    # --- CASO 4 ---
    print("--- CASO 4: Estimación de Gastos ---")
    input_4 = "¿Cuánto sería el gasto aproximado por persona, pidiendo los tres platos?"
    print(f"👤 Usuario: {input_4}")

    chat_history.append(HumanMessage(content=input_4))
    response_4 = agent_graph.invoke({"messages": chat_history})

    chat_history = response_4["messages"]
    imprimir_respuesta(response_4)

    print("-" * 50)
    print("🔍 ESTADO FINAL DE LA BASE DE DATOS:")
    print(f"Reservas guardadas: {DB_RESERVAS['reservas']}")
    print(f"Disponibilidad restante: {DB_RESERVAS['mesas_disponibles']}")

/tmp/ipython-input-1542123833.py:75: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent_graph = create_react_agent(


🚀 AGENTE INICIADO (Gemini 2.5 + LangGraph + LangChain)

--- CASO 1: Consulta de Menú ---
👤 Usuario: Hola, ¿tienen opciones veganas?
🤖 Agente: Sí, tenemos opciones veganas bajo petición.

--- CASO 2: Intento Reserva (22:00 Lleno) ---
👤 Usuario: Quiero reservar para 4 personas a las 22:00.
🤖 Agente: Lo siento, a las 22:00 no tenemos disponibilidad. Las horas libres son: 19:00, 20:00, 21:00. ¿Te gustaría reservar en alguna de esas horas?

--- CASO 3: Corrección (21:00 Disponible) ---
👤 Usuario: Entendido, entonces a las 21:00 a nombre de Ana.
🤖 Agente: ¡Perfecto! Tu reserva para 4 personas a las 21:00 a nombre de Ana ha sido confirmada. ¡Esperamos verte pronto!

--- CASO 4: Estimación de Gastos ---
👤 Usuario: ¿Cuánto sería el gasto aproximado por persona, pidiendo los tres platos?
🤖 Agente: El gasto aproximado por persona, pidiendo una entrada, un plato principal y un postre, sería entre 45€ y 49€, dependiendo de los platos elegidos.

--------------------------------------------------
🔍 E

# Recursos
* [API de Gemini](https://ai.google.dev/api?hl=es-419)
* [Pydantic](https://docs.pydantic.dev/latest/)
* [LangChain](https://docs.langchain.com/)